**Exercise 2.67.**

*Below is about Huffman encoding trees*

It includes: 

    (symbol & weight) pair list(data) -> generate-huffman-tree (procedure) -> huffman-encoding-tree (data)
    list of bits & tree(data) -> decode (procedure) -> message (data)
    message & tree(data) -> encode (procedure) -> list of bits (data)

Exercises are complements of that in textbook.

2.67 is to let you try the decoding procedure. 2.68 is to design a encode procedure. 2.69 & 2.70 is about generate-huffman-tree.  

(a d a b b c a)

In [30]:
(define sample-tree
  (make-code-tree (make-leaf 'A 4)
                  (make-code-tree
                   (make-leaf 'B 2)
                   (make-code-tree (make-leaf 'D 1)
                                   (make-leaf 'C 1)))))

(define sample-message '(0 1 1 0 0 1 0 1 0 1 1 1 0))

sample-message

In [19]:
(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits)
        '()
        (let ((next-branch
               (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch)
              (cons (symbol-leaf next-branch)
                    (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch)))))
  (decode-1 bits tree))
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit -- CHOOSE-BRANCH" bit))))
(decode sample-message sample-tree)

(a d a b b c a)

In [17]:
; Predefined

(define (make-leaf symbol weight)
  (list 'leaf symbol weight))
(define (leaf? object)
  (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))
(define (make-code-tree left right)
  (list left
        right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))))

(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree)
      (list (symbol-leaf tree))
      (caddr tree)))
(define (weight tree)
  (if (leaf? tree)
      (weight-leaf tree)
      (cadddr tree)))

weight

**Exercise 2.68.**

(a d a b b c a) =>  '(0 1 1 0 0 1 0 1 0 1 1 1 0)

In [28]:
(define (encode message tree)
  (if (null? message)
      '()
      (append (encode-symbol (car message) tree)
              (encode (cdr message) tree))))

(define (encode-symbol str t)
  (cond ((or (null? t) (not (tree-has-symbol? str t))) (error " the symbol is not in the tree at all" t))
        ((leaf? t) '())
        ((tree-has-symbol? str (left-branch t)) (cons 0 (encode-symbol str (left-branch t))))
        (else (cons 1 (encode-symbol str (right-branch t))))))
(define (tree-has-symbol? s t)
  (memq s (symbols t)))

tree-has-symbol?

In [31]:
(encode '(a d a b b c a) sample-tree)

(0 1 1 0 0 1 0 1 0 1 1 1 0)

**Exercise 2.69.**



In [5]:
;predefined
(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set)
                    (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs)
      '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair)    ; symbol
                               (cadr pair))  ; frequency
                    (make-leaf-set (cdr pairs))))))

make-leaf-set

In [34]:
(make-leaf-set '())

()

In [36]:
(define (generate-huffman-tree pairs)
  (successive-merge (make-leaf-set pairs)))

(define (successive-merge ordered-leaves)
  (cond ((null? ordered-leaves) '())
        ((= (length ordered-leaves) 1) (car ordered-leaves))
        (else (let ((new-tree (make-code-tree (car ordered-leaves) (cadr ordered-leaves)))
                    (rest-leaves (cddr ordered-leaves)))
                (successive-merge (adjoin-set new-tree rest-leaves))))))


successive-merge

**Exercise 2.70.** 

84 bits is needed.

Using fixed-length for 8 symbols, each needs 3bits. Totally: 3 X (2 + 16 + 1+3+2+9+2+1) = 108

In [3]:
;symbol-frequency pairs 
(define songs '((A 2) (NA 16) (BOOM 1) (SHA 3) (GET 2) (YIP 9) (JOB 2) (WAH 1)))
(make-leaf-set songs)
(generate-huffman-tree songs)

unbound-variable: Unbound variable: make-leaf-set

In [4]:
(encode '(Sha boom) (generate-huffman-tree songs))

unbound-variable: Unbound variable: generate-huffman-tree

    (1 1 1 1 1 1 1 0 0 1 1 1 1 0) ;Get a job
    (1 1 1 0 0 0 0 0 0 0 0 0);Sha na na na na na na na na
    (1 1 1 1 1 1 1 0 0 1 1 1 1 0) ;Get a job
    (1 1 1 0 0 0 0 0 0 0 0 0) ;Sha na na na na na na na na
    (1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0);Wah yip yip yip yip yip yip yip yip yip
    (1 1 1 0 1 1 0 1 1) ;Sha boom

**Exercise 2.71.** 

n = 5, the depth of tree is 4, 4 bits are required for encode the most frequent symbol. The least: 1 bits.

**Exercise 2.72.** 

the most frequent: search the symbol list: n, search how many node: 1    O(n)

the least frequent: search the symbol list: n,n-1,n-2, n-3 ...        O(n^2)